In [2]:
from statistics import stdev
from os import walk, makedirs
import pandas as pd
import numpy as np

### Create results files for each fold of every for each model

In [3]:

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

predictions_dir = '../predictions'
# k_folds_dirs = next(walk(predictions_dir))[1]
pretrained_dirnames =['bert-base-multilingual-cased-sentiment-multilingual',
                      'dehatebert-mono-french',
                      'french_xlm_xnli',
                      'xlm-roberta-base-sentiment-multilingual']

result_cols=["model", "traindata", "accuracy", "std_accuracy", "precision", "std_precision", "recall", "std_recall", "f1", "std_f1"]
dfs ={'bert-base-multilingual-cased-sentiment-multilingual':pd.DataFrame(columns=result_cols),
      'dehatebert-mono-french':pd.DataFrame(columns=result_cols),
      'french_xlm_xnli':pd.DataFrame(columns=result_cols),
      'xlm-roberta-base-sentiment-multilingual':pd.DataFrame(columns=result_cols)}

for pretrained in pretrained_dirnames:
  df = dfs.get(pretrained)
  result_files = next(walk(f"{predictions_dir}/k_is_0/{pretrained}/"))[2]
  for file in result_files: # 'text', 'class', 'predicted'
    prec = list()
    recall = list()
    f1 = list()
    accuracy = list()
    for k in range(0,3):
      results = pd.read_csv(f"{predictions_dir}/k_is_{k}/{pretrained}/{file}").sample(frac=1)
      prec_curr, recall_curr, f1_curr, _ = precision_recall_fscore_support(results["class"], results["predicted"], average='macro')
      accuracy_curr = accuracy_score(results["class"], results["predicted"])
      prec.append(prec_curr)
      recall.append(recall_curr)
      f1.append(f1_curr)
      accuracy.append(accuracy_curr)
    
    # add results to dict
    df.loc[len(df.index)] = [pretrained, file,
                            np.mean(accuracy), stdev(accuracy),
                            np.mean(prec), stdev(prec),
                            np.mean(recall), stdev(recall),
                            np.mean(f1), stdev(f1)]

In [4]:
results_dir="../results"
for pretrained in pretrained_dirnames:
  dfs[pretrained].to_csv(f"{results_dir}/{pretrained}", index=False)

In [3]:
# pred0 = pd.read_csv(f"../predictions/k_is_0/french_xlm_xnli/1epoch_train28").sample(frac=1)
# pred1 = pd.read_csv(f"../predictions/k_is_1/french_xlm_xnli/1epoch_train28").sample(frac=1)
# pred2 = pd.read_csv(f"../predictions/k_is_2/french_xlm_xnli/1epoch_train28").sample(frac=1)

              model       traindata  accuracy  std_accuracy  precision  \
25  french_xlm_xnli   4epoch_train2  0.910714      0.043052   0.913224   
11  french_xlm_xnli   3epoch_train4  0.934524      0.032295   0.940691   
26  french_xlm_xnli  1epoch_train28  0.961310      0.006819   0.961480   
17  french_xlm_xnli  5epoch_train28  0.964286      0.004464   0.964659   
12  french_xlm_xnli  2epoch_train22  0.965774      0.006819   0.966475   
23  french_xlm_xnli   3epoch_train2  0.903274      0.044717   0.907965   
15  french_xlm_xnli   5epoch_train4  0.934524      0.034097   0.939050   
5   french_xlm_xnli  4epoch_train16  0.965774      0.002577   0.966580   
18  french_xlm_xnli   5epoch_train8  0.944940      0.010310   0.946867   
3   french_xlm_xnli   1epoch_train4  0.918155      0.043360   0.920127   
0   french_xlm_xnli  5epoch_train22  0.970238      0.002577   0.971364   
16  french_xlm_xnli   2epoch_train4  0.936012      0.034676   0.941061   
13  french_xlm_xnli   3epoch_train8  0